<a href="https://colab.research.google.com/github/didulabhanuka/colab/blob/main/tomato_ripeness_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Required Libraries

In [ ]:
!pip install ultralytics albumentations torchvision effdet pycocotools

Import Libraries and Mount Google Drive

In [ ]:
import os
import cv2
import zipfile
from albumentations import Compose, HorizontalFlip, Rotate, RandomBrightnessContrast, HueSaturationValue
from ultralytics import YOLO
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from effdet import create_model
from effdet.bench import DetBenchTrain
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import functional as F

from google.colab import drive
drive.mount('/content/drive')


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


Unzip Dataset

In [ ]:
dataset_zip = "/content/drive/MyDrive/tomato_ripeness_classifier/tomato_dataset.zip"
dataset_dir = "/content/dataset/dataset"

os.makedirs("/content/dataset", exist_ok=True)
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")

print("Dataset unzipped successfully.")


Dataset unzipped successfully.


Apply Augmentations

In [ ]:
augmented_dir = "/content/dataset_augmented"

# Create Augmented Dataset Folders
os.makedirs(f"{augmented_dir}/train/images", exist_ok=True)
os.makedirs(f"{augmented_dir}/train/labels", exist_ok=True)
os.makedirs(f"{augmented_dir}/val/images", exist_ok=True)
os.makedirs(f"{augmented_dir}/val/labels", exist_ok=True)

def augment_data(image_folder, label_folder, output_image_folder, output_label_folder):
    # Augmentation Pipeline with Brightness and Hue-Saturation Adjustments
    augmentations = Compose(
        [
            RandomBrightnessContrast(p=0.2),
            HueSaturationValue(p=0.2),
        ],
        bbox_params={"format": "yolo", "label_fields": ["class_labels"]}
    )

    for image_file in os.listdir(image_folder):
        img_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + ".txt")
        image = cv2.imread(img_path)
        h, w, _ = image.shape

        # Read and Validate Bounding Boxes
        with open(label_path, "r") as f:
            bboxes = []
            class_labels = []
            for line in f.readlines():
                cls, x_center, y_center, width, height = map(float, line.strip().split())

                # Validate the input bounding boxes
                if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                    print(f"Invalid bbox in {image_file}: {[x_center, y_center, width, height]}")
                    continue

                bboxes.append([x_center, y_center, width, height])
                class_labels.append(int(cls))

        if not bboxes:
            print(f"No valid bounding boxes found for {image_file}. Skipping augmentation.")
            continue

        # Debug: Log input bounding boxes
        print(f"Processing {image_file}: {bboxes}")

        try:
            # Apply Augmentations
            augmented = augmentations(image=image, bboxes=bboxes, class_labels=class_labels)
            augmented_image = augmented["image"]
            augmented_bboxes = augmented["bboxes"]
            augmented_class_labels = augmented["class_labels"]

            # Clip Bounding Boxes to Ensure Valid Values After Augmentation
            clipped_bboxes = []
            for bbox in augmented_bboxes:
                x_center, y_center, width, height = bbox
                x_center = min(max(x_center, 0), 1)
                y_center = min(max(y_center, 0), 1)
                width = min(max(width, 0), 1)
                height = min(max(height, 0), 1)
                clipped_bboxes.append([x_center, y_center, width, height])

            # Save Augmented Image
            output_img_path = os.path.join(output_image_folder, image_file)
            cv2.imwrite(output_img_path, augmented_image)

            # Save Updated Labels
            output_label_path = os.path.join(output_label_folder, os.path.splitext(image_file)[0] + ".txt")
            with open(output_label_path, "w") as f:
                for bbox, cls in zip(clipped_bboxes, augmented_class_labels):
                    x_center, y_center, width, height = bbox
                    f.write(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        except Exception as e:
            print(f"Error processing {image_file}: {e}")
            continue

    print(f"Augmented data saved to {output_image_folder} and {output_label_folder}.")

# Apply Augmentation to Train and Validation Sets
augment_data(
    image_folder=f"{dataset_dir}/train/images",  # Corrected path
    label_folder=f"{dataset_dir}/train/labels",  # Corrected path
    output_image_folder=f"{augmented_dir}/train/images",
    output_label_folder=f"{augmented_dir}/train/labels"
)

augment_data(
    image_folder=f"{dataset_dir}/val/images",  # Corrected path
    label_folder=f"{dataset_dir}/val/labels",  # Corrected path
    output_image_folder=f"{augmented_dir}/val/images",
    output_label_folder=f"{augmented_dir}/val/labels"
)


/usr/local/lib/python3.10/dist-packages/albumentations/core/composition.py:205: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


Processing IMG_20191215_112544.jpg: [[0.3465, 0.345, 0.2492, 0.1799], [0.6202, 0.2961, 0.2585, 0.2035], [0.6551, 0.4352, 0.1374, 0.1088], [0.5581, 0.471, 0.2137, 0.1533], [0.3011, 0.5588, 0.2026, 0.1446], [0.5488, 0.6217, 0.1683, 0.121], [0.5022, 0.7211, 0.1229, 0.097], [0.4091, 0.6606, 0.0984, 0.0712], [0.3447, 0.6536, 0.0804, 0.0668], [0.3683, 0.7176, 0.117, 0.076], [0.3567, 0.7687, 0.1019, 0.076], [0.0285, 0.6392, 0.0571, 0.045], [0.1118, 0.6643, 0.0478, 0.0341], [0.2344, 0.7818, 0.0891, 0.0734], [0.4388, 0.7547, 0.0693, 0.0629], [0.3161, 0.8314, 0.044, 0.0353], [0.4511, 0.8101, 0.0401, 0.0403], [0.1675, 0.8061, 0.0806, 0.0608], [0.1785, 0.8403, 0.0656, 0.0472], [0.0719, 0.5865, 0.0623, 0.0533], [0.4929, 0.5461, 0.0914, 0.0563], [0.6082, 0.5619, 0.0635, 0.0249]]
Error processing IMG_20191215_112544.jpg: Expected x_min for bbox [-5.0001e-05      0.6167     0.05705      0.6617           5] to be in the range [0.0, 1.0], got -5.000084638595581e-05.
Processing IMG_20191215_111425.jpg: [

Create YAML Configuration


In [ ]:
yaml_content = f"""
train: {augmented_dir}/train/images
val: {augmented_dir}/val/images

nc: 6
names: ["b_fully_ripened", "b_half_ripened", "b_green", "l_fully_ripened", "l_half_ripened", "l_green"]
"""
with open("tomato_ripeness_classifier.yaml", "w") as yaml_file:
    yaml_file.write(yaml_content)

print("Dataset YAML file created.")


Dataset YAML file created.


Train YOLOv8


In [ ]:
yolo_model = YOLO('yolov8m.pt')  # Medium model for better accuracy
yolo_model.train(data="tomato_ripeness_classifier.yaml", epochs=50, imgsz=640)


100%|██████████| 49.7M/49.7M [00:00<00:00, 338MB/s]


Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=tomato_ripeness_classifier.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

100%|██████████| 755k/755k [00:00<00:00, 81.0MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 285MB/s]


...

                   all        153       1899      0.775      0.763      0.814      0.706



50 epochs completed in 1.083 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:12<00:00,  2.53s/it]


                   all        153       1899      0.762      0.771      0.828      0.721
       b_fully_ripened         38         68      0.729       0.75      0.772      0.687
        b_half_ripened         47        103      0.715      0.757      0.807      0.713
               b_green         67        367      0.885      0.877      0.949      0.829
       l_fully_ripened         56        255      0.703        0.8       0.82      0.714
        l_half_ripened         64        210      0.673      0.608      0.714      0.631
               l_green         66        896      0.867      0.837      0.907       0.75
Speed: 0.5ms preprocess, 9.7ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4bd50ee170>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

Train Faster R-CNN


In [ ]:
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

class CustomDataset(Dataset):
    def __init__(self, image_folder, label_folder):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.image_files = os.listdir(image_folder)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        img_path = os.path.join(self.image_folder, image_file)
        label_path = os.path.join(self.label_folder, os.path.splitext(image_file)[0] + ".txt")

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        with open(label_path, "r") as f:
            boxes = []
            labels = []
            for line in f.readlines():
                cls, x_center, y_center, width, height = map(float, line.strip().split())
                xmin = (x_center - width / 2) * w
                ymin = (y_center - height / 2) * h
                xmax = (x_center + width / 2) * w
                ymax = (y_center + height / 2) * h
                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(int(cls))

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image = F.to_tensor(image)

        target = {"boxes": boxes, "labels": labels}

        return image, target

# Custom Collate Function to Handle Variable Number of Bounding Boxes
def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)

# Create DataLoader for Faster R-CNN
train_dataset = CustomDataset(f"{augmented_dir}/train/images", f"{augmented_dir}/train/labels")
train_loader = DataLoader(
    train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn
)

# Load Faster R-CNN with COCO weights
weights = FasterRCNN_ResNet50_FPN_Weights.COCO_V1
frcnn_model = fasterrcnn_resnet50_fpn(weights=weights)

# Modify the classifier for 7 classes (including background)
in_features = frcnn_model.roi_heads.box_predictor.cls_score.in_features
frcnn_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 7)

optimizer = torch.optim.SGD(frcnn_model.parameters(), lr=0.005, momentum=0.9)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
frcnn_model.to(device)
frcnn_model.train()

# Training Loop
for epoch in range(10):  # Adjust the number of epochs as needed
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = frcnn_model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {losses.item()}")


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 165MB/s]
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: roi_align_backward_kernel does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1, Loss: 0.6034991145133972
Epoch 2, Loss: 0.4442107677459717
Epoch 3, Loss: 0.47171857953071594
Epoch 4, Loss: 0.28911569714546204
Epoch 5, Loss: 0.20760910212993622
Epoch 6, Loss: 0.4430273771286011
Epoch 7, Loss: 0.19285368919372559
Epoch 8, Loss: 0.2252819687128067
Epoch 9, Loss: 0.10515951365232468
Epoch 10, Loss: 0.07782146334648132


Ensemble Predictions

In [ ]:
def ensemble_predictions(yolo_preds, frcnn_preds):
    final_preds = []
    for i in range(len(yolo_preds)):
        votes = [yolo_preds[i], frcnn_preds[i]]
        final_preds.append(max(set(votes), key=votes.count))  # Majority voting
    return final_preds

# Example usage
yolo_preds = [1, 2, 2, 3]
frcnn_preds = [1, 2, 3, 3]

ensemble_preds = ensemble_predictions(yolo_preds, frcnn_preds)
print("Ensemble Predictions:", ensemble_preds)


Ensemble Predictions: [1, 2, 2, 3]


YOLOv8 Validation

In [ ]:
results = yolo_model.val()
print("YOLOv8 Validation Results:", results)


NameError: name 'yolo_model' is not defined